In [1]:
import numpy as np
import pandas as pd
from confluent_kafka import Producer
from connect import get_connection
from random import randint
from simulation import transaktion_factory
from time import sleep
import socket


from confluent_kafka import Consumer

# Einlesen des Buchbestands

In [2]:
df_bestand = pd.read_csv('data/Bestand.csv', sep=';', encoding='unicode_escape')

In [3]:
bestand_cols=df_bestand.columns

# Einlesen der Tabellen des Producers

In [4]:
c = Consumer({
    'bootstrap.servers': '0.0.0.0:9092',
    'group.id': socket.gethostname(),
    'auto.offset.reset': 'latest'
})


In [5]:
prod_Transaktion_cols={
    0:'Aktion',
    1:'Datum',
    2:'ID_Exemplar',
    3:'ID_Kunde',
    4:'Titel',
    5:'Autor',
    6:'Jahr',
    7:'Art',
    8:'Kennung',
    9:'Zugriffsort'
}

prod_Bewertung_cols={
    0:'Aktion',
    1:'ID_Kunde',
    2:'ID_Buch',
    3:'Wertung',
    4:'Rezension'
}

prod_Neukunden_cols={
    0:'Aktion',
    1:'ID_Kunde',
    2:'Kundennr',
    3:'Vorname_1',
    4:'Vorname_2',
    5:'Nachname',
    6:'Anrede',
    7:'PLZ',
    8:'Strasse',
    9:'Hausnr',
   10:'Mail',
   11:'Tel',
   12:'Geschlecht',
   13:'Geburtsdatum',
   14:'Beruf',
   15:'Titel',
   16:'PersoNr',
   17:'PersoValidTo' 
}

cols=[prod_Transaktion_cols,prod_Bewertung_cols,prod_Neukunden_cols]
topics=["Transaktion", "Bewertung", "Neukunden"]

dfd={}

In [6]:
for col, topic in zip(cols, topics):
    print(col, topic)

    dfd[topic] = pd.DataFrame(columns=list(col.values()))
    
    
    c.subscribe([topic])
    
    for i in range(60):
        msg = c.poll(1)
        if msg is None:
            continue
    
        mrow = pd.DataFrame([msg.value().decode('utf-8').split(";")]).rename(columns=col)

        dfd[topic]=pd.concat([dfd[topic], mrow])

    dfd[topic] = dfd[topic].reset_index(drop=True)
    #c.close()

{0: 'Aktion', 1: 'Datum', 2: 'ID_Exemplar', 3: 'ID_Kunde', 4: 'Titel', 5: 'Autor', 6: 'Jahr', 7: 'Art', 8: 'Kennung', 9: 'Zugriffsort'} Transaktion
{0: 'Aktion', 1: 'ID_Kunde', 2: 'ID_Buch', 3: 'Wertung', 4: 'Rezension'} Bewertung
{0: 'Aktion', 1: 'ID_Kunde', 2: 'Kundennr', 3: 'Vorname_1', 4: 'Vorname_2', 5: 'Nachname', 6: 'Anrede', 7: 'PLZ', 8: 'Strasse', 9: 'Hausnr', 10: 'Mail', 11: 'Tel', 12: 'Geschlecht', 13: 'Geburtsdatum', 14: 'Beruf', 15: 'Titel', 16: 'PersoNr', 17: 'PersoValidTo'} Neukunden


In [7]:
dfd['Transaktion']

,Aktion,Datum,ID_Exemplar,ID_Kunde,Titel,Autor,Jahr,Art,Kennung,Zugriffsort
0,Rückgabe,2024-12-06 07:00:00,131,6952,Der Geliebte der Mutter,Urs Widmer,2000.0,Roman,EKv83725,sVv47569
1,Leihe,2024-12-06 10:00:00,214,5539,Spaziergang nach Syrakus im Jahre 1802,Johann Gottfried Seume,1803.0,Roman,PlB90283,ZlQ31749
2,Leihe,2024-12-06 13:00:00,255,4796,Satyricon,Petronius,60.0,Roman,IPB24689,LbN63028
3,Leihe,2024-12-06 13:00:00,145,5764,"Leben und Ansichten von Tristram Shandy, Gent...",Laurence Sterne,1759.0,Roman,xgD90687,eAt08743
4,Leihe,2024-12-06 13:00:00,280,5278,Sozaboy,Ken Saro-Wiwa,1985.0,Roman,wJZ25810,UWw29435
5,Leihe,2024-12-06 16:00:00,239,518,Eine Geschichte der Welt in 10 ½ Kapiteln,Julian Barnes,1989.0,Roman,YQh18763,YOU49601
6,Leihe,2024-12-06 16:00:00,129,5194,"Der Mann, der seine Frau mit einem Hut verwec...",Oliver Sacks,1985.0,Sachbuch,sKJ91358,qNa13509
7,Leihe,2024-12-06 16:00:00,136,4676,Metamorphosen,Ovid,8.0,Lyrik,nZA29518,eBP01286
8,Leihe,2024-12-06 18:00:00,115,2935,Oden,Horaz,-13.0,Lyrik,Pca90157,ZCi47160
9,Leihe,2024-12-06 18:00:00,105,4666,Farm der Tiere,George Orwell,1946.0,Roman,Qib96713,iXA67895


In [8]:
dfd['Bewertung']

,Aktion,ID_Kunde,ID_Buch,Wertung,Rezension
0,Bewertung,148,274,5,Das Buch war weder besonders beeindruckend no...
1,Bewertung,243,486,6,Das Buch pendelte zwischen interessanten Ansä...
2,Bewertung,32,948,5,Das Buch war durchschnittlich. Einige Teile w...


In [9]:
dfd['Neukunden']

,Aktion,ID_Kunde,Kundennr,Vorname_1,Vorname_2,Nachname,Anrede,PLZ,Strasse,Hausnr,Mail,Tel,Geschlecht,Geburtsdatum,Beruf,Titel,PersoNr,PersoValidTo
0,Neukunde,302,7541,Henry,nan,Huber,Herr,72147,Meisenweg,2.0,henry.huber@gmx.net,07473/462095,m,28.11.1924,Senior/in,None,CGL63587,20.02.2023


# Cleaning der Tabellen

### Transaktion

In [10]:
# IDs zu int konvertieren
dfd['Transaktion']['ID_Exemplar'] = pd.to_numeric(dfd['Transaktion']['ID_Exemplar']).astype(int)
dfd['Transaktion']['ID_Kunde'] = pd.to_numeric(dfd['Transaktion']['ID_Kunde']).astype(int)
# Jahre zu int konvertieren
dfd['Transaktion']['Jahr'] = pd.to_numeric(dfd['Transaktion']['Jahr']).astype(int)

### Bewertung

In [11]:
# IDs zu int konvertieren
dfd['Bewertung']['ID_Kunde'] = pd.to_numeric(dfd['Bewertung']['ID_Kunde']).astype(int)
dfd['Bewertung']['ID_Buch'] = pd.to_numeric(dfd['Bewertung']['ID_Buch']).astype(int)
dfd['Bewertung']['Wertung'] = pd.to_numeric(dfd['Bewertung']['Wertung']).astype(int)

### Neukunden

In [12]:
# IDs zu int konvertieren
dfd['Neukunden']['ID_Kunde'] = pd.to_numeric(dfd['Neukunden']['ID_Kunde']).astype(int)
dfd['Neukunden']['Kundennr'] = pd.to_numeric(dfd['Neukunden']['Kundennr']).astype(int)

# Anreden konsistent machen 
dfd['Neukunden']['Anrede'] = dfd['Neukunden']['Anrede'].replace(['Fr\.'],'Frau', regex=True)
dfd['Neukunden']['Anrede'] = dfd['Neukunden']['Anrede'].replace(['Hr\.'],'Herr', regex=True)
# Titel: "None" to NaN
dfd['Neukunden']['Titel']  = np.where(dfd['Neukunden']['Titel'].str.contains("None"), np.nan, dfd['Neukunden']['Titel'] )
# Vorname_2: "nan" to NaN
dfd['Neukunden']['Vorname_2']  = np.where(dfd['Neukunden']['Vorname_2'].str.contains("nan"), np.nan, dfd['Neukunden']['Vorname_2'] )
# Hausnr: "nan" to NaN
dfd['Neukunden']['Hausnr']  = np.where(dfd['Neukunden']['Hausnr'].str.contains("nan"), np.nan, dfd['Neukunden']['Hausnr'] )
# 4-stellige PLZ mit "0" präfixen 
dfd['Neukunden']['PLZ']    = dfd['Neukunden']['PLZ'].str.strip().str.rjust(5, '0')

# Hausnummern aus Strasse-Spalte extrahieren
def extract_hausnr(strasse):
  if pd.isna(strasse):
    return np.nan
  parts = strasse.split()
  if len(parts) > 1 and parts[-1].isdigit():
    return parts[-1]
  else:
    return np.nan
#Apply extract_hausnr, wenn 'Hausnr' NaN und entferne die Hausnummern aus der Strasse Spalte
dfd['Neukunden'].loc[dfd['Neukunden']['Hausnr'].isnull(), 'Hausnr'] = dfd['Neukunden'].loc[dfd['Neukunden']['Hausnr'].isnull(), 'Strasse'].apply(extract_hausnr)
dfd['Neukunden']['Strasse'] = dfd['Neukunden']['Strasse'].str.rsplit(expand=True)[0]
# Convert Hausnr to int
dfd['Neukunden']['Hausnr'] = pd.to_numeric(dfd['Neukunden']['Hausnr']).astype(int)

# Geschlecht konsistent machen 
dfd['Neukunden']['Geschlecht'] = dfd['Neukunden']['Geschlecht'].str.strip().replace(["^w$", "^W$"],'Weiblich', regex=True)
dfd['Neukunden']['Geschlecht'] = dfd['Neukunden']['Geschlecht'].str.strip().replace(["^m$", "^M$"],'Männlich', regex=True)

# Datumsspalten konsistent machen
dfd['Neukunden']['Geburtsdatum'] = pd.to_datetime(dfd['Neukunden']['Geburtsdatum'],format='mixed', dayfirst=True)
dfd['Neukunden']['PersoValidTo'] = pd.to_datetime(dfd['Neukunden']['PersoValidTo'],format='mixed', dayfirst=True)

### Bestand

In [13]:
# Jahre zu int konvertieren
df_bestand['Jahr'] = pd.to_numeric(df_bestand['Jahr']).astype('Int64', errors='ignore')

# Aufbau der Tabellen für das Core DWH

In [14]:
dfs={}

In [15]:
Kunde_cols={
    0:'ID_Kunde',
    1:'ID_Mitgliedsstatus',
    2:'Kundennr',
    3:'Vorname_1',
    4:'Vorname_2',
    5:'Nachname',
    6:'Anrede',
    7:'Titel',
    8:'PLZ',
    9:'Strasse',
   10:'Hausnr',
   11:'Mail',
   12:'Tel',
   13:'Geschlecht',
   14:'Geburtsdatum',
   15:'Beruf',
   16:'PersoNr',
   17:'PersoValidTo', 
   18:'Mitglied seit',
   19:'Mitglied bis' 
}

Bewertung_cols={
    1:'ID_Kunde',
    2:'ID_Buch',
    3:'Wertung',
    4:'Rezension'
}

Buch_cols={
    1:'ID_Kunde',
    2:'ID_Buch',
    3:'Wertung',
    4:'Rezension'
}

Exemplare_cols={
    1:'ID_Bestand',
    2:'Nr',
    3:'Kennung',
    4:'Zugriffsort',
    5:'Zustand'
}

Buch_cols={
    1:'ID_Autor',
    2:'Nr',
    3:'Titel',
    4:'Jahr',
    5:'Art',
}

Leihe_cols={
    0:'ID_Kunde',
    1:'ID_Exemplar',
    2:'Ausleihdatum',
    3:'Rueckgabedatum',
    4:'Verlaengerungsstatus',
    5:'Mahnstatus',
    6:'Fernleihe'
}

Mitgliedsstatus_cols={
    0:'ID_Mitgliedsstatus',
    2:'Bezeichnung',
    3:'Jahresbeitrag',
    4:'Mahnbetrag'
}

Beitragszahlung_cols={
    0:'ID_Zahlung',
    2:'ID_Kunde',
    3:'Datum',
    4:'Betrag',
}

In [16]:
dfs['Kunde']     = dfd['Neukunden'].reindex(columns=list(Kunde_cols.values()))
dfs['Bewertung'] = dfd['Bewertung'].reindex(columns=list(Bewertung_cols.values()))
dfs['Exemplare'] = df_bestand.reindex(columns=list(Exemplare_cols.values())).rename(columns={
                                                                        'ID_Bestand':'ID_Exemplar',
                                                                        'Nr':'ID_Buch'    
                                                                        })

dfs['Autor']   = df_bestand[['Autor', 'Herkunft']].drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index':'ID_Autor'})

dfs['Buch']    = pd.merge(df_bestand, 
                          dfs['Autor'], 
                          on=['Autor']
                         )[list(Buch_cols.values())]                                  \
                                                    .drop_duplicates()                \
                                                    .reset_index(drop=True)           \
                                                    .rename(columns={'Nr':'ID_Buch'})

dfs['Leihe'] = dfd['Transaktion'].reindex(columns=list(Leihe_cols.values())).reset_index().rename(columns={'index':'ID_Leihe'})


### more ###
dfs['Mitgliedsstatus'] = pd.DataFrame(columns=list(Mitgliedsstatus_cols.values()))
dfs['Beitragszahlung'] = dfd['Neukunden'].reindex(columns=list(Beitragszahlung_cols.values()))


%4|1727338159.294|MAXPOLL|rdkafka#consumer-1| [thrd:main]: Application maximum poll interval (300000ms) exceeded by 20ms (adjust max.poll.interval.ms for long-running message processing): leaving group


# Schicke die Tabellen an den SQL Server


In [36]:
from sqlalchemy import create_engine, inspect, text

USER = 'root'
PASSWORD = 'root'
HOST = 'localhost'          # see yml file
PORT = 3307
DATABASE = 'test_db_1'
 

def get_connection(database=DATABASE):
    return create_engine(
        url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}"\
            .format(USER, PASSWORD, HOST, PORT, database)
    )

engine = get_connection('mysql')
with engine.connect() as connection:
    insp = inspect(engine)
    db_list = insp.get_schema_names()
    print(f"Connection to the {HOST} for user {USER} created successfully.")
    if DATABASE not in db_list:
        sql = text(f"CREATE DATABASE {DATABASE} CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")
        result = connection.execute(sql)
        if result:
            print(f"Database {DATABASE} created!")

Connection to the localhost for user root created successfully.


In [37]:
for Tabelle in dfs.keys():
    dfs[Tabelle].to_sql(Tabelle, con=engine,schema=DATABASE, index=False, if_exists='append') 